In [1]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import h5py

OSError: [WinError 1114] Eine DLL-Initialisierungsroutine ist fehlgeschlagen. Error loading "c:\Users\piecu\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

## Task 1: CNNs for Galaxy Classification

In [ ]:
# create data folder if it does not exist
import os
os.makedirs("data", exist_ok=True)

import urllib.request
_, msg = urllib.request.urlretrieve(
    "http://www.astro.utoronto.ca/~bovy/Galaxy10/Galaxy10.h5", 
    "data/Galaxy10.h5"
)

In [ ]:
label_names = [
    'Disk, Face-on, No Spiral',
    'Smooth, Completely round',
    'Smooth, in-between round',
    'Smooth, Cigar shaped',
    'Disk, Edge-on, Rounded Bulge',
    'Disk, Edge-on, Boxy Bulge',
    'Disk, Edge-on, No Bulge',
    'Disk, Face-on, Tight Spiral',
    'Disk, Face-on, Medium Spiral',
    'Disk, Face-on, Loose Spiral'
]
n_classes = len(label_names)

# To get the images and labels from file
with h5py.File('data/Galaxy10.h5', 'r') as F:
    images = np.array(F['images'])
    labels = np.array(F['ans'])
images = images.astype(np.float32)

# comply to (batch, channel, height, width) convention of pytorch
images = np.moveaxis(images, -1, 1)  
# convert to torch
images = torch.from_numpy(images)
labels = torch.from_numpy(labels)

print(f'{images.shape=}, {labels.shape=}')

### (a)

In [ ]:
# TODO: plot three samples of each class


In [ ]:
from torchvision.transforms import Normalize

# TODO: Split the data and normalize the images:

# TODO: Create tensordatasets and data loaders:

### (b)

In [ ]:
#TODO: implement a small CNN as specified on the sheet
from torch import nn

class GalaxyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        ...

    def forward(self, x):
        ...
        return x

### 

### (c)

In [ ]:
# TODO: Instantiate the model, optimizer and criterion
model = GalaxyCNN()

# optimizer = ?
# criterion = ?

### (d)

In [ ]:

train_losses = []
val_losses = []
val_accs = []

# TODO: Implement the training loop, validating after every epoch, and make the requested plots.

def validate(model):
    losses = []
    correct = []

    #TODO: Implement the validation loop
    
    avg_loss = np.mean(np.array(losses))
    accuracy = np.mean(np.array(correct))
    print(f'{accuracy=:.2f}, {avg_loss=:.2e}')
    return avg_loss, accuracy

val_loss, val_acc = validate(model)
val_losses.append(val_loss)
val_accs.append(val_acc)
for epoch in range(30):
    # TODO: Implement the training loop, validating after every epoch and a visialization of the loss curves
    ...

### (e)

In [ ]:
# TODO: Plot the confusion matrix

### (f)

In [ ]:
# TODO: Plot the convolutional kernels of the first CNN layer